# Modelos de Ecuaciones Estructurales (SEM) - Variables Filtradas

Este notebook implementa **modelos de path analysis** para analizar la relación entre las variables ideológicas y las variables de cambio de opinión/tiempo **filtradas** (solo ítems significativos).

## Modelo para cada variable dependiente:

```
Indice_Progresismo ────→ Y
                          ↑
Indice_Conservadurismo ──→
```

**Ecuación:**
```
Y = β₀ + β₁(Indice_Progresismo) + β₂(Indice_Conservadurismo) + ε
```

## Variables Dependientes (8 modelos):

**Cambio de Opinión Filtrado (CO):**
1. `Cambio_Op_Filt_Pro_Izq`
2. `Cambio_Op_Filt_Pro_Der`
3. `Cambio_Op_Filt_Con_Izq`
4. `Cambio_Op_Filt_Con_Der`

**Cambio de Tiempo Filtrado (CT):**
5. `Cambio_Tiempo_Filt_Pro_Izq`
6. `Cambio_Tiempo_Filt_Pro_Der`
7. `Cambio_Tiempo_Filt_Con_Izq`
8. `Cambio_Tiempo_Filt_Con_Der`

## Diferencia con Variables Sumadas:

Las **variables filtradas** solo incluyen los ítems que fueron **significativos** en el análisis de Kruskal-Wallis (p < 0.05), por lo que esperamos:
- **Mayor R²** (mejor ajuste del modelo)
- **Coeficientes más fuertes** (relaciones más claras)
- **Mayor significancia estadística**

In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Imports para SEM
try:
    from semopy import Model
    from semopy.inspector import inspect
    print("✓ semopy disponible")
    USAR_SEMOPY = True
except ImportError:
    print("⚠️  semopy no disponible, usando statsmodels OLS")
    from statsmodels.api import OLS, add_constant
    from sklearn.preprocessing import StandardScaler
    USAR_SEMOPY = False

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

⚠️  semopy no disponible, usando statsmodels OLS
✓ Librerías cargadas exitosamente


## 1. Cargar Datos

In [2]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

✓ Datos cargados:
  - Generales: 2786 registros
  - Ballotage: 1254 registros


## 2. Definir Variables

In [3]:
# Variables predictoras
Predictores = [
    'Indice_Progresismo',
    'Indice_Conservadurismo'
]

# Variables dependientes - Filtradas
Outcomes_Filtradas = [
    'Cambio_Op_Filt_Pro_Izq',
    'Cambio_Op_Filt_Pro_Der',
    'Cambio_Op_Filt_Con_Izq',
    'Cambio_Op_Filt_Con_Der',
    'Cambio_Tiempo_Filt_Pro_Izq',
    'Cambio_Tiempo_Filt_Pro_Der',
    'Cambio_Tiempo_Filt_Con_Izq',
    'Cambio_Tiempo_Filt_Con_Der'
]

print(f"✓ Variables definidas:")
print(f"  - Predictores: {len(Predictores)}")
print(f"  - Outcomes: {len(Outcomes_Filtradas)}")

✓ Variables definidas:
  - Predictores: 2
  - Outcomes: 8


## 3. Función para Ejecutar Modelo SEM

In [4]:
def Ejecutar_Modelo_Path(df, outcome, predictores, nombre_modelo):
    """
    Ejecuta un modelo de path analysis (regresión múltiple).
    
    Parámetros:
    -----------
    df : DataFrame
        Datos
    outcome : str
        Variable dependiente
    predictores : list
        Lista de predictores
    nombre_modelo : str
        Nombre del modelo (para reportes)
    
    Retorna:
    --------
    dict : Resultados del modelo
    """
    
    # Verificar que las variables existan
    variables_necesarias = [outcome] + predictores
    variables_existentes = [v for v in variables_necesarias if v in df.columns]
    
    if len(variables_existentes) != len(variables_necesarias):
        faltantes = set(variables_necesarias) - set(variables_existentes)
        print(f"  ⚠️  Variables faltantes en {nombre_modelo}: {faltantes}")
        return None
    
    # Seleccionar datos y eliminar NaN
    df_modelo = df[variables_necesarias].dropna()
    
    if len(df_modelo) < 10:
        print(f"  ⚠️  Datos insuficientes en {nombre_modelo}: n={len(df_modelo)}")
        return None
    
    if USAR_SEMOPY:
        # Usar semopy para SEM verdadero
        try:
            # Definir modelo en sintaxis semopy
            modelo_spec = f"{outcome} ~ {' + '.join(predictores)}"
            
            modelo = Model(modelo_spec)
            modelo.fit(df_modelo)
            
            # Extraer resultados
            params = modelo.inspect()
            
            # Calcular R²
            residuos = df_modelo[outcome] - modelo.predict(df_modelo)[outcome]
            ss_res = np.sum(residuos**2)
            ss_tot = np.sum((df_modelo[outcome] - df_modelo[outcome].mean())**2)
            r_squared = 1 - (ss_res / ss_tot)
            
            # Obtener métricas de ajuste
            try:
                stats_modelo = inspect(modelo)
                aic = stats_modelo.get('AIC', np.nan)
                bic = stats_modelo.get('BIC', np.nan)
            except:
                aic = np.nan
                bic = np.nan
            
            # Formatear resultados
            resultados = {
                'Modelo': nombre_modelo,
                'Outcome': outcome,
                'n': len(df_modelo),
                'R²': r_squared,
                'AIC': aic,
                'BIC': bic,
                'Coeficientes': {}
            }
            
            # Extraer coeficientes
            for idx, row in params.iterrows():
                if row['lval'] == outcome and row['rval'] in predictores:
                    resultados['Coeficientes'][row['rval']] = {
                        'β': row['Estimate'],
                        'SE': row['Std. Err'],
                        'z': row['z-value'],
                        'p': row['p-value']
                    }
            
            return resultados
            
        except Exception as e:
            print(f"  ❌ Error en semopy para {nombre_modelo}: {e}")
            print(f"     Usando regresión OLS como respaldo...")
    
    # Fallback: Usar OLS de statsmodels
    try:
        from statsmodels.api import OLS, add_constant
        
        # Preparar datos
        X = df_modelo[predictores]
        y = df_modelo[outcome]
        
        # Agregar constante
        X_const = add_constant(X)
        
        # Ajustar modelo
        modelo_ols = OLS(y, X_const).fit()
        
        # Calcular coeficientes estandarizados
        from sklearn.preprocessing import StandardScaler
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_std = scaler_X.fit_transform(X)
        y_std = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
        
        modelo_std = OLS(y_std, X_std).fit()
        
        # Formatear resultados
        resultados = {
            'Modelo': nombre_modelo,
            'Outcome': outcome,
            'n': len(df_modelo),
            'R²': modelo_ols.rsquared,
            'AIC': modelo_ols.aic,
            'BIC': modelo_ols.bic,
            'Coeficientes': {}
        }
        
        # Extraer coeficientes (excluyendo constante)
        for i, pred in enumerate(predictores):
            resultados['Coeficientes'][pred] = {
                'β': modelo_ols.params[pred],
                'β_std': modelo_std.params[i],
                'SE': modelo_ols.bse[pred],
                't': modelo_ols.tvalues[pred],
                'p': modelo_ols.pvalues[pred]
            }
        
        return resultados
        
    except Exception as e:
        print(f"  ❌ Error en OLS para {nombre_modelo}: {e}")
        return None

## 4. Ejecutar Modelos para GENERALES

In [5]:
print("="*70)
print("EJECUTANDO MODELOS SEM: GENERALES (Variables Filtradas)")
print("="*70)

resultados_generales = []

for outcome in Outcomes_Filtradas:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Generales,
        outcome,
        Predictores,
        f"Generales_{outcome}"
    )
    
    if resultado:
        resultados_generales.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  AIC = {resultado['AIC']:.2f}")
        print(f"  BIC = {resultado['BIC']:.2f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', 0))
            p_val = coefs.get('p', 1)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  (p = {p_val:.4f}) {sig}")
        
        print(f"\n  {'✅ Modelo ajustado exitosamente' if resultado['R²'] > 0 else '⚠️  R² muy bajo'}")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_generales)} modelos ejecutados para Generales")
print("="*70)

EJECUTANDO MODELOS SEM: GENERALES (Variables Filtradas)

📊 Modelo: Cambio_Op_Filt_Pro_Izq
----------------------------------------------------------------------
  n = 2786
  R² = 0.0108
  AIC = 9447.49
  BIC = 9465.29

  Coeficientes:
    Indice_Progresismo        β = -0.2378  (p = 0.0000) ***
    Indice_Conservadurismo    β = -0.2427  (p = 0.0000) ***

  ✅ Modelo ajustado exitosamente

📊 Modelo: Cambio_Op_Filt_Pro_Der
----------------------------------------------------------------------
  n = 2786
  R² = nan
  AIC = -inf
  BIC = -inf

  Coeficientes:
    Indice_Progresismo        β =  0.0000  (p = nan) ns
    Indice_Conservadurismo    β =  0.0000  (p = nan) ns

  ⚠️  R² muy bajo

📊 Modelo: Cambio_Op_Filt_Con_Izq
----------------------------------------------------------------------
  n = 2786
  R² = 0.0316
  AIC = 9247.45
  BIC = 9265.25

  Coeficientes:
    Indice_Progresismo        β = -0.1013  (p = 0.0163) *
    Indice_Conservadurismo    β = -0.4841  (p = 0.0000) ***

  ✅ Modelo a

## 5. Ejecutar Modelos para BALLOTAGE

In [6]:
print("="*70)
print("EJECUTANDO MODELOS SEM: BALLOTAGE (Variables Filtradas)")
print("="*70)

resultados_ballotage = []

for outcome in Outcomes_Filtradas:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Ballotage,
        outcome,
        Predictores,
        f"Ballotage_{outcome}"
    )
    
    if resultado:
        resultados_ballotage.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  AIC = {resultado['AIC']:.2f}")
        print(f"  BIC = {resultado['BIC']:.2f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', 0))
            p_val = coefs.get('p', 1)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  (p = {p_val:.4f}) {sig}")
        
        print(f"\n  {'✅ Modelo ajustado exitosamente' if resultado['R²'] > 0 else '⚠️  R² muy bajo'}")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_ballotage)} modelos ejecutados para Ballotage")
print("="*70)

EJECUTANDO MODELOS SEM: BALLOTAGE (Variables Filtradas)

📊 Modelo: Cambio_Op_Filt_Pro_Izq
----------------------------------------------------------------------
  n = 1254
  R² = 0.0192
  AIC = 3473.73
  BIC = 3489.13

  Coeficientes:
    Indice_Progresismo        β = -0.1565  (p = 0.0007) ***
    Indice_Conservadurismo    β =  0.0245  (p = 0.6675) ns

  ✅ Modelo ajustado exitosamente

📊 Modelo: Cambio_Op_Filt_Pro_Der
----------------------------------------------------------------------
  n = 1254
  R² = 0.0227
  AIC = 3663.29
  BIC = 3678.69

  Coeficientes:
    Indice_Progresismo        β = -0.2552  (p = 0.0000) ***
    Indice_Conservadurismo    β = -0.1324  (p = 0.0316) *

  ✅ Modelo ajustado exitosamente

📊 Modelo: Cambio_Op_Filt_Con_Izq
----------------------------------------------------------------------
  n = 1254
  R² = 0.0370
  AIC = 3905.83
  BIC = 3921.23

  Coeficientes:
    Indice_Progresismo        β = -0.0391  (p = 0.4766) ns
    Indice_Conservadurismo    β = -0.3806  

## 6. Crear Tablas de Resultados

In [7]:
def Crear_Tabla_Resultados(lista_resultados, nombre_dataset):
    """
    Convierte lista de resultados en DataFrames para exportar.
    
    Retorna:
    --------
    df_metricas : DataFrame con R², AIC, BIC
    df_coeficientes : DataFrame con β y p-valores
    """
    
    # Tabla de métricas de ajuste
    metricas_data = []
    
    for res in lista_resultados:
        metricas_data.append({
            'Outcome': res['Outcome'],
            'n': res['n'],
            'R²': res['R²'],
            'AIC': res['AIC'],
            'BIC': res['BIC']
        })
    
    df_metricas = pd.DataFrame(metricas_data)
    
    # Tabla de coeficientes
    coef_data = []
    
    for res in lista_resultados:
        for pred, coefs in res['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', np.nan))
            p_val = coefs.get('p', np.nan)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            coef_data.append({
                'Outcome': res['Outcome'],
                'Predictor': pred,
                'β': beta,
                'SE': coefs.get('SE', np.nan),
                'p-valor': p_val,
                'Sig': sig
            })
    
    df_coeficientes = pd.DataFrame(coef_data)
    
    return df_metricas, df_coeficientes

In [8]:
# Crear tablas para Generales
df_metricas_gen, df_coef_gen = Crear_Tabla_Resultados(resultados_generales, 'Generales')

print("\n📋 TABLA DE MÉTRICAS - GENERALES:")
print(df_metricas_gen.to_string(index=False))

print("\n📋 TABLA DE COEFICIENTES - GENERALES:")
print(df_coef_gen.to_string(index=False))


📋 TABLA DE MÉTRICAS - GENERALES:
                   Outcome    n       R²          AIC          BIC
    Cambio_Op_Filt_Pro_Izq 2786 0.010827  9447.489010  9465.286097
    Cambio_Op_Filt_Pro_Der 2786      NaN         -inf         -inf
    Cambio_Op_Filt_Con_Izq 2786 0.031642  9247.452303  9265.249390
    Cambio_Op_Filt_Con_Der 2786 0.019763  7949.154221  7966.951308
Cambio_Tiempo_Filt_Pro_Izq 2786 0.002280 18873.327723 18891.124809
Cambio_Tiempo_Filt_Pro_Der 2786 0.006312 22375.507224 22393.304311
Cambio_Tiempo_Filt_Con_Izq 2786 0.000381 24548.807502 24566.604588
Cambio_Tiempo_Filt_Con_Der 2786 0.002801 25104.012210 25121.809296

📋 TABLA DE COEFICIENTES - GENERALES:
                   Outcome              Predictor         β       SE      p-valor Sig
    Cambio_Op_Filt_Pro_Izq     Indice_Progresismo -0.237839 0.043705 5.727683e-08 ***
    Cambio_Op_Filt_Pro_Izq Indice_Conservadurismo -0.242656 0.058067 3.019697e-05 ***
    Cambio_Op_Filt_Pro_Der     Indice_Progresismo  0.000000 0.00000

In [9]:
# Crear tablas para Ballotage
df_metricas_bal, df_coef_bal = Crear_Tabla_Resultados(resultados_ballotage, 'Ballotage')

print("\n📋 TABLA DE MÉTRICAS - BALLOTAGE:")
print(df_metricas_bal.to_string(index=False))

print("\n📋 TABLA DE COEFICIENTES - BALLOTAGE:")
print(df_coef_bal.to_string(index=False))


📋 TABLA DE MÉTRICAS - BALLOTAGE:
                   Outcome    n       R²         AIC         BIC
    Cambio_Op_Filt_Pro_Izq 1254 0.019217 3473.728808 3489.131089
    Cambio_Op_Filt_Pro_Der 1254 0.022737 3663.291255 3678.693536
    Cambio_Op_Filt_Con_Izq 1254 0.037050 3905.830834 3921.233115
    Cambio_Op_Filt_Con_Der 1254 0.006637 2826.488849 2841.891130
Cambio_Tiempo_Filt_Pro_Izq 1254      NaN        -inf        -inf
Cambio_Tiempo_Filt_Pro_Der 1254      NaN        -inf        -inf
Cambio_Tiempo_Filt_Con_Izq 1254 0.008345 9033.263140 9048.665422
Cambio_Tiempo_Filt_Con_Der 1254 0.002746 9153.707213 9169.109495

📋 TABLA DE COEFICIENTES - BALLOTAGE:
                   Outcome              Predictor         β       SE      p-valor Sig
    Cambio_Op_Filt_Pro_Izq     Indice_Progresismo -0.156489 0.046226 7.330610e-04 ***
    Cambio_Op_Filt_Pro_Izq Indice_Conservadurismo  0.024523 0.057080 6.675373e-01  ns
    Cambio_Op_Filt_Pro_Der     Indice_Progresismo -0.255247 0.049856 3.539539e-07 ***

## 7. Guardar Resultados en Excel

In [10]:
# Crear carpeta de resultados
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')
if not os.path.exists(Carpeta_Resultados):
    os.makedirs(Carpeta_Resultados)

# Guardar Generales
archivo_gen = os.path.join(Carpeta_Resultados, 'SEM_Variables_Filtradas_Generales.xlsx')
with pd.ExcelWriter(archivo_gen, engine='openpyxl') as writer:
    df_metricas_gen.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_gen.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ Resultados Generales guardados: SEM_Variables_Filtradas_Generales.xlsx")

# Guardar Ballotage
archivo_bal = os.path.join(Carpeta_Resultados, 'SEM_Variables_Filtradas_Ballotage.xlsx')
with pd.ExcelWriter(archivo_bal, engine='openpyxl') as writer:
    df_metricas_bal.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_bal.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ Resultados Ballotage guardados: SEM_Variables_Filtradas_Ballotage.xlsx")

✓ Resultados Generales guardados: SEM_Variables_Filtradas_Generales.xlsx
✓ Resultados Ballotage guardados: SEM_Variables_Filtradas_Ballotage.xlsx


## 8. Comparación: Filtradas vs Sumadas

In [11]:
print("="*70)
print("COMPARACIÓN: VARIABLES FILTRADAS vs SUMADAS")
print("="*70)

# Cargar resultados de variables sumadas si existen
archivo_sumadas_gen = os.path.join(Carpeta_Resultados, 'SEM_Variables_Sumadas_Generales.xlsx')
archivo_sumadas_bal = os.path.join(Carpeta_Resultados, 'SEM_Variables_Sumadas_Ballotage.xlsx')

if os.path.exists(archivo_sumadas_gen):
    df_sumadas_gen = pd.read_excel(archivo_sumadas_gen, sheet_name='Métricas de Ajuste')
    
    print("\n📊 GENERALES - Comparación de R²:")
    print("-"*70)
    print(f"\n  Variables Sumadas (promedio): {df_sumadas_gen['R²'].mean():.4f}")
    print(f"  Variables Filtradas (promedio): {df_metricas_gen['R²'].mean():.4f}")
    print(f"  Mejora: {(df_metricas_gen['R²'].mean() - df_sumadas_gen['R²'].mean()):.4f}")
    
    if df_metricas_gen['R²'].mean() > df_sumadas_gen['R²'].mean():
        print(f"  ✅ Variables filtradas explican MÁS varianza")
    else:
        print(f"  ⚠️  Variables sumadas explican más varianza")
else:
    print("\n⚠️  No se encontraron resultados de Variables Sumadas para comparar")

if os.path.exists(archivo_sumadas_bal):
    df_sumadas_bal = pd.read_excel(archivo_sumadas_bal, sheet_name='Métricas de Ajuste')
    
    print("\n📊 BALLOTAGE - Comparación de R²:")
    print("-"*70)
    print(f"\n  Variables Sumadas (promedio): {df_sumadas_bal['R²'].mean():.4f}")
    print(f"  Variables Filtradas (promedio): {df_metricas_bal['R²'].mean():.4f}")
    print(f"  Mejora: {(df_metricas_bal['R²'].mean() - df_sumadas_bal['R²'].mean()):.4f}")
    
    if df_metricas_bal['R²'].mean() > df_sumadas_bal['R²'].mean():
        print(f"  ✅ Variables filtradas explican MÁS varianza")
    else:
        print(f"  ⚠️  Variables sumadas explican más varianza")

print("\n" + "="*70)

COMPARACIÓN: VARIABLES FILTRADAS vs SUMADAS

📊 GENERALES - Comparación de R²:
----------------------------------------------------------------------

  Variables Sumadas (promedio): 0.0181
  Variables Filtradas (promedio): 0.0106
  Mejora: -0.0076
  ⚠️  Variables sumadas explican más varianza

📊 BALLOTAGE - Comparación de R²:
----------------------------------------------------------------------

  Variables Sumadas (promedio): 0.0209
  Variables Filtradas (promedio): 0.0161
  Mejora: -0.0047
  ⚠️  Variables sumadas explican más varianza



## 9. Identificar Mejores Modelos

In [12]:
print("="*70)
print("MEJORES MODELOS (por R²) - VARIABLES FILTRADAS")
print("="*70)

print("\n🏆 Top 3 Modelos - GENERALES:")
print("-"*70)
top3_gen = df_metricas_gen.sort_values('R²', ascending=False).head(3)
print(top3_gen[['Outcome', 'R²', 'AIC', 'BIC']].to_string(index=False))

print("\n🏆 Top 3 Modelos - BALLOTAGE:")
print("-"*70)
top3_bal = df_metricas_bal.sort_values('R²', ascending=False).head(3)
print(top3_bal[['Outcome', 'R²', 'AIC', 'BIC']].to_string(index=False))

print("\n" + "="*70)

MEJORES MODELOS (por R²) - VARIABLES FILTRADAS

🏆 Top 3 Modelos - GENERALES:
----------------------------------------------------------------------
               Outcome       R²         AIC         BIC
Cambio_Op_Filt_Con_Izq 0.031642 9247.452303 9265.249390
Cambio_Op_Filt_Con_Der 0.019763 7949.154221 7966.951308
Cambio_Op_Filt_Pro_Izq 0.010827 9447.489010 9465.286097

🏆 Top 3 Modelos - BALLOTAGE:
----------------------------------------------------------------------
               Outcome       R²         AIC         BIC
Cambio_Op_Filt_Con_Izq 0.037050 3905.830834 3921.233115
Cambio_Op_Filt_Pro_Der 0.022737 3663.291255 3678.693536
Cambio_Op_Filt_Pro_Izq 0.019217 3473.728808 3489.131089



## 10. Resumen Final

In [13]:
print("="*70)
print("RESUMEN FINAL: MODELOS SEM - VARIABLES FILTRADAS")
print("="*70)

print("\n📊 Modelos ejecutados:")
print(f"  - Generales: {len(resultados_generales)} modelos")
print(f"  - Ballotage: {len(resultados_ballotage)} modelos")
print(f"  - Total: {len(resultados_generales) + len(resultados_ballotage)} modelos")

print("\n📈 Estadísticas generales:")
print(f"  Generales - R² promedio: {df_metricas_gen['R²'].mean():.4f}")
print(f"  Ballotage - R² promedio: {df_metricas_bal['R²'].mean():.4f}")

# Contar coeficientes significativos
sig_gen = len(df_coef_gen[df_coef_gen['Sig'] != 'ns'])
sig_bal = len(df_coef_bal[df_coef_bal['Sig'] != 'ns'])

print(f"\n  Generales - Coeficientes significativos: {sig_gen}/{len(df_coef_gen)}")
print(f"  Ballotage - Coeficientes significativos: {sig_bal}/{len(df_coef_bal)}")

print("\n📁 Archivos generados:")
print("  - SEM_Variables_Filtradas_Generales.xlsx")
print("  - SEM_Variables_Filtradas_Ballotage.xlsx")

print("\n🎯 Ventaja de Variables Filtradas:")
print("  - Solo incluyen ítems significativos (p<0.05 en Kruskal-Wallis)")
print("  - Mayor poder predictivo (esperamos R² más alto)")
print("  - Relaciones más claras entre ideología y cambio")

print("\n" + "="*70)
print("✅ ANÁLISIS SEM COMPLETADO")
print("="*70)

RESUMEN FINAL: MODELOS SEM - VARIABLES FILTRADAS

📊 Modelos ejecutados:
  - Generales: 8 modelos
  - Ballotage: 8 modelos
  - Total: 16 modelos

📈 Estadísticas generales:
  Generales - R² promedio: 0.0106
  Ballotage - R² promedio: 0.0161

  Generales - Coeficientes significativos: 8/16
  Ballotage - Coeficientes significativos: 5/16

📁 Archivos generados:
  - SEM_Variables_Filtradas_Generales.xlsx
  - SEM_Variables_Filtradas_Ballotage.xlsx

🎯 Ventaja de Variables Filtradas:
  - Solo incluyen ítems significativos (p<0.05 en Kruskal-Wallis)
  - Mayor poder predictivo (esperamos R² más alto)
  - Relaciones más claras entre ideología y cambio

✅ ANÁLISIS SEM COMPLETADO
